In [1]:
import numpy as np

from bokeh.io import output_notebook, show
from bokeh.plotting import figure
from bokeh.models import Slider, Button

from bokeh.layouts import row, widgetbox

from bokeh.application.handlers import FunctionHandler
from bokeh.application import Application

output_notebook()

Loading BokehJS ...

In [2]:
g = 9.81

v0_init = 3
w0_init = 35
alpha_deg_init = 70
alpha_init = alpha_deg_init*np.pi/180
Npoly_init = 3

t_fin = 1.5

FPS = 120
dt = 1/FPS
SPEED_init=1

X_RANGE = 2 #en mètres
X_WIDTH = 850 #en pixels
ASPECT_RATIO = 1.9 #rapport largeur sur hauteur






#Npoly = 4

r = 0.04
theta0 = np.pi/6

#w = 35



def x(t, v0, alpha):
    return v0*np.cos(alpha)*t

def y(t, v0, alpha):
    return -1/2 *g*t**2 + v0*np.sin(alpha)*t

from bokeh.layouts import column, row
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, Select, TextInput
from time import sleep, time







def modify_doc(doc):
    
    v0 = v0_init
    w0 = w0_init
    alpha=alpha_init
    alpha_deg = alpha_deg_init
    Npoly = Npoly_init
    
    SPEED = SPEED_init
    
    def get_data(v0, w0, alpha, Npoly, t_fin, SPEED=SPEED_init):
        data_dict = {}
        data_dict['t'] = np.arange(0,t_fin,SPEED*dt)
        data_dict['x'] = x(data_dict['t'], v0, alpha)
        data_dict['y'] = y(data_dict['t'], v0, alpha)

        #Coordoneées des coins du polygone, à partir de celles du centre de gravité
        for i in range(Npoly):
            data_dict['x_point'+str(i)] = data_dict['x'] + r*np.cos(i*(2*np.pi/Npoly) + w0*data_dict['t'])
            data_dict['y_point'+str(i)] = data_dict['y'] + r*np.sin(i*(2*np.pi/Npoly) + w0*data_dict['t'])


        return data_dict

    def get_data_poly(data_dict, Npoly):
        poly_data_dict = {}

        current_points_x = [ data_dict['x_point'+str(i)][-1] for i in range(Npoly) ]
        current_points_y = [ data_dict['y_point'+str(i)][-1] for i in range(Npoly) ]

        poly_data_dict['x'] = current_points_x
        poly_data_dict['y'] = current_points_y
        return poly_data_dict
    
    
        
    
    
    
    def create_main_fig_and_source(v0, w0, alpha, Npoly, t_fin):
        global p, source, source_poly
        
        data_dict = get_data(v0=v0, w0=w0, alpha=alpha, Npoly=Npoly, t_fin=t_fin)

        source = ColumnDataSource( data=data_dict )
        source_poly = ColumnDataSource( data=get_data_poly(data_dict, Npoly) )
        
        
        ##p = figure(x_range=[0,1.5], y_range=[0,.7], plot_width=700, plot_height=300, tools="", toolbar_location=None)
        #p = figure(x_range=[0,1.5], y_range=[0,.7], plot_width=700, plot_height=300, tools="", toolbar_location=None)
        p = figure(x_range=[0,X_RANGE], y_range=[0,X_RANGE/ASPECT_RATIO], plot_width=int(X_WIDTH),
                   plot_height=int(X_WIDTH/ASPECT_RATIO), x_axis_label='x [m]', y_axis_label='y [m]',
                   tools="", toolbar_location=None)
        
        #Trajectoire du centre de gravité
        l = p.line(x='x', y='y', source=source, line_width=2)
        
        #Trajectoire de chaque coin du polygone
        for i in range(Npoly):
            p.line(x='x_point'+str(i), y='y_point'+str(i), source=source, line_width=2, alpha=.6, color='brown')
        
        #Affichage du polygone en bout de trajectoire.
        p.patch(x='x', y='y', source=source_poly, fill_color='red', fill_alpha=.3, line_color='purple')
         
    create_main_fig_and_source(v0, w0, alpha, Npoly, t_fin)
    
    v0_slider = Slider(title="V₀", start=2, end=6, step=.01, value=v0)
    w0_slider = Slider(title="ω₀", start=-50, end=50, step=.1, value=w0)
    alpha_deg_slider = Slider(title="α", start=1, end=89, step=.01, value=alpha_deg)
    Npoly_slider = Slider(title="Npoly", start=2, end=12, step=1, value=Npoly)
    
    SPEED_slider = Slider(title="Vitesse de lecture", start=.1, end=SPEED_init, step=.05, value=SPEED, format='0 %')

    
    def update_points(attrname, old, new):
        v0 = v0_slider.value
        w0 = w0_slider.value
        alpha_deg = alpha_deg_slider.value
        alpha = alpha_deg*np.pi/180
        Npoly = int(Npoly_slider.value)
        
        
        data_dict = get_data(v0, w0, alpha, Npoly, t_fin)
        source.data = data_dict
        source_poly.data = get_data_poly(data_dict, Npoly)
        
    def refresh_Npoly(attrname, old, new):
        v0 = v0_slider.value
        w0 = w0_slider.value
        alpha_deg = alpha_deg_slider.value
        alpha = alpha_deg*np.pi/180
        Npoly = int(Npoly_slider.value)
        
        create_main_fig_and_source(v0, w0, alpha, Npoly, t_fin)
        
        layout.children[-1].children[0] = p
        
    
    v0_slider.on_change('value', update_points)
    w0_slider.on_change('value', update_points) 
    alpha_deg_slider.on_change('value', update_points)
    Npoly_slider.on_change('value', refresh_Npoly)    

    
    
    play_button = Button(label="Play")
    
    def play_animation():
        t = 0
        t_index = 0
        
        v0 = v0_slider.value
        w0 = w0_slider.value
        alpha_deg = alpha_deg_slider.value
        alpha = alpha_deg*np.pi/180
        Npoly = int(Npoly_slider.value)
        
        SPEED = SPEED_slider.value
        
        
        # Précalcule toute l'animation ; ensuite à chaque passage de la boucle d'animation ci-dessous,
        # seules les données du prochain timestep seront rajoutées dans l'objet de données dynamique "source.data".
        

        full_anim_data = get_data(v0, w0, alpha, Npoly, t_fin, SPEED=SPEED)
        
        last_t_index = len(full_anim_data['t']) - 1
        
        
        # Initialise le dict displayed_data en lui insérant que les valeurs à t=0 de toutes les données.
        displayed_data = {}
        for key in full_anim_data:
            displayed_data[key] = full_anim_data[key][0:1]       
        source.data = displayed_data
        source_poly.data = get_data_poly(displayed_data, Npoly)
        
        is_in_flight = True
        all_previous_vertices_in_midair = False
        
        ##Ne semple plus néccessaire
        sleep(.2) #Sinon on ne voit pas toujours le début de l'animation
        
        
        #while t <= t_fin - SPEED*dt:
        while t_index < last_t_index :
            sleep(dt)
            t += SPEED*dt
            t_index += 1
            
            for key in displayed_data:
                displayed_data[key] =  full_anim_data[key][:t_index+1]
                
            source.data = displayed_data
            source_poly.data = get_data_poly(displayed_data, Npoly)
            
            
            
            
            #Conditions d'interruption de l'animation (lorsque l'objet ratterit).
            
            current_poly_ys = np.array( source_poly.data['y'] )
            if np.any(current_poly_ys <= 0):
                if all_previous_vertices_in_midair:
                    # Cas 'normal' où tous les points du solide ont été dans l'air (ordonnée > 0)
                    # au moins un instant
                    # --> arrêt de l'animation dès qu'un d'entre eux repasse sous le sol.
                    break
                elif np.all(current_poly_ys <= 0):
                    # Cas des lancers où l'objet frole le sol et il n'y a aucun instant
                    # où tous les points du solide ont été dans l'air
                    # --> arrêt de l'animation lorsqu'ils sont tous repassés sous le sol.
                    break
                
            
            
            
            if np.all(current_poly_ys > 0):
                all_previous_vertices_in_midair = True
        
    
    
    play_button.on_click(play_animation)
    

    
    row1 = row(alpha_deg_slider, v0_slider, w0_slider)
    row2 = row(Npoly_slider, SPEED_slider)

    row_play = row(play_button)
    row_fig = row(p) 



    layout = column( row1, row2 , row_play, row_fig)

    
    
    #layout = column(row(v0_slider, w0_slider, alpha_deg_slider, Npoly_slider, width=200), row(play_button), row(p))
    doc.add_root(layout)


#show(modify_doc)

# Set up the Application 
handler = FunctionHandler(modify_doc)
app = Application(handler)

# Create the Document
# Not strictly necessary, but helps w/ debugging
doc = app.create_document()

# Show the application
# Make sure the URL matches your Jupyter instance
show(app, notebook_url="localhost:8888")